<a href="https://colab.research.google.com/github/mpekar1/DSS-IDI-modelling/blob/main/Markov_Chain_modelling_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Clean and Transform Data

In [1]:
library(readxl)
library(dplyr)
library(stringr)
library( tidyverse)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks base::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Load the transition counts and population data
transition_counts <- read_excel("Markov Chain Outputs 2024_06_13_corrected_for_release.xlsx", sheet = "Transition counts 2011-20 conf", skip=1)
transition_counts_2021 <- read_excel("Markov Chain Outputs 2024_06_13_corrected_for_release.xlsx", sheet = "Transition counts 2021 conf",skip=1)
dss_pop_by_age <- read_excel("Markov Chain Outputs 2024_06_13_corrected_for_release.xlsx", sheet = "Total DSS pop by age conf")
new_entrants <- read_excel("Markov Chain Outputs 2024_06_13_corrected_for_release.xlsx", sheet = "New entrants conf")


In [3]:
## Inspect the data

head(transition_counts)
head(transition_counts_2021)
head(dss_pop_by_age)
head(new_entrants)

age_groups,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,NOT BORN YET,OT,OTHER,OVERSEAS,T1,Grand Total
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Age 0-04,DECEASED,12,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,12
Age 0-04,DSS,15,7854,S,S,S,S,186,S,⋯,S,S,6,426,S,S,S,21,S,8532
Age 0-04,DSS_EMPLOYED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,6
Age 0-04,DSS_HOSPITAL,9,411,S,S,S,S,231,S,⋯,S,S,18,42,S,S,S,S,S,732
Age 0-04,DSS_T2,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S
Age 0-04,DSS-OTHER,S,27,S,S,S,S,S,S,⋯,S,S,S,S,S,9,S,S,S,138


age_groups,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EDU_EMPLOYED,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,OT,OTHER,OVERSEAS,T1,Grand Total
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Age 0-04,DECEASED,9,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,9
Age 0-04,DSS,S,1311,S,S,S,S,12,S,⋯,S,S,S,S,93,S,S,S,S,1422
Age 0-04,DSS_EMPLOYED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S
Age 0-04,DSS_HOSPITAL,S,48,S,S,S,S,15,S,⋯,S,S,S,S,S,S,S,S,S,72
Age 0-04,DSS-OTHER,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,18
Age 0-04,EMPLOYED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S


age_group,refyr,total_pop
<chr>,<dbl>,<dbl>
Age 0-04,2010,1101
Age 0-04,2011,1062
Age 0-04,2012,975
Age 0-04,2013,900
Age 0-04,2014,915
Age 0-04,2015,966


age_group,refyr,Sum of new_entrants
<chr>,<dbl>,<dbl>
Age 0-04,2010,1098
Age 0-04,2011,462
Age 0-04,2012,420
Age 0-04,2013,390
Age 0-04,2014,444
Age 0-04,2015,516


In [6]:
transition_counts <- transition_counts %>%
  rename(age_group = age_groups)

transition_counts_2021 <- transition_counts_2021 %>%
  rename(age_group = age_groups)

In [7]:
names(transition_counts)
names(transition_counts_2021)
names(dss_pop_by_age)
names(new_entrants)

[1] "age_group"       "from_state"      "DECEASED"        "DSS"            
 [5] "DSS_EDU"         "DSS_EDU_T1"      "DSS_EMPLOYED"    "DSS_EMPLOYED_T1"
 [9] "DSS_HOSPITAL"    "DSS_T1"          "DSS_T2"          "DSS-OTHER"      
[13] "EDU"             "EDU_EMPLOYED"    "EMPLOYED"        "EMPLOYED_T1"    
[17] "HOSPITAL"        "NO DATA"         "NOT BORN YET"    "OT"             
[21] "OTHER"           "OVERSEAS"        "T1"              "Grand Total"

[1] "age_group"       "from_state"      "DECEASED"        "DSS"            
 [5] "DSS_EDU"         "DSS_EDU_T1"      "DSS_EMPLOYED"    "DSS_EMPLOYED_T1"
 [9] "DSS_HOSPITAL"    "DSS_T1"          "DSS_T2"          "DSS-OTHER"      
[13] "EDU"             "EDU_EMPLOYED"    "EMPLOYED"        "EMPLOYED_T1"    
[17] "HOSPITAL"        "NO DATA"         "OT"              "OTHER"          
[21] "OVERSEAS"        "T1"              "Grand Total"

[1] "age_group" "refyr"     "total_pop"

[1] "age_group"           "refyr"               "Sum of new_entrants"

In [8]:
# Check the column names of the transition_counts dataframe
print(names(transition_counts))

 [1] "age_group"       "from_state"      "DECEASED"        "DSS"            
 [5] "DSS_EDU"         "DSS_EDU_T1"      "DSS_EMPLOYED"    "DSS_EMPLOYED_T1"
 [9] "DSS_HOSPITAL"    "DSS_T1"          "DSS_T2"          "DSS-OTHER"      
[13] "EDU"             "EDU_EMPLOYED"    "EMPLOYED"        "EMPLOYED_T1"    
[17] "HOSPITAL"        "NO DATA"         "NOT BORN YET"    "OT"             
[21] "OTHER"           "OVERSEAS"        "T1"              "Grand Total"    


In [13]:
# Function to clean transition counts data
clean_transition_data <- function(df) {
  df %>%
    mutate(across(-c(age_group, from_state), ~ replace(., . == "S", 0))) %>%
    mutate(across(-c(age_group, from_state), as.numeric)) %>%
    filter(!grepl("Total", age_group)) %>%
    select(-`Grand Total`)
}

# Function to clean DSS population and new entrants data
clean_population_data <- function(df, col_to_clean) {
  df %>%
    mutate(across(all_of(col_to_clean), ~ replace(., . == "S", 0))) %>%
    mutate(across(all_of(col_to_clean), as.numeric))
}

# Apply the function to clean the DSS population by age and new entrants data
dss_pop_by_age <- clean_population_data(dss_pop_by_age, "total_pop")
new_entrants <- clean_population_data(new_entrants, "new_entrants")

# Verify the results
head(transition_counts)
head(transition_counts_2021)
head(dss_pop_by_age)
head(new_entrants)


age_group,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EDU_EMPLOYED,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,NOT BORN YET,OT,OTHER,OVERSEAS,T1
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Age 0-04,DECEASED,12,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,DSS,15,7854,0,0,0,0,186,0,⋯,0,0,0,6,426,0,0,0,21,0
Age 0-04,DSS_EMPLOYED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,DSS_HOSPITAL,9,411,0,0,0,0,231,0,⋯,0,0,0,18,42,0,0,0,0,0
Age 0-04,DSS_T2,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,DSS-OTHER,0,27,0,0,0,0,0,0,⋯,0,0,0,0,0,0,9,0,0,0


age_group,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EDU,EDU_EMPLOYED,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,OT,OTHER,OVERSEAS,T1
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Age 0-04,DECEASED,9,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,DSS,0,1311,0,0,0,0,12,0,⋯,0,0,0,0,0,93,0,0,0,0
Age 0-04,DSS_EMPLOYED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,DSS_HOSPITAL,0,48,0,0,0,0,15,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,DSS-OTHER,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-04,EMPLOYED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


## Step 2: Calculate Historical Transition Probabilities


1. Compute Transition Counts and Probabilities:

In [14]:
# Transform the transition counts data to a long format
transition_counts_long <- transition_counts %>%
  pivot_longer(cols = -c(age_group, from_state), names_to = "to_state", values_to = "count") %>%
  filter(!is.na(count))

transition_counts_2021_long <- transition_counts_2021 %>%
  pivot_longer(cols = -c(age_group, from_state), names_to = "to_state", values_to = "count") %>%
  filter(!is.na(count))

# Compute transition probabilities
transition_probabilities <- transition_counts_long %>%
  group_by(age_group, from_state) %>%
  summarise(total_count = sum(count), .groups = 'drop') %>%
  left_join(transition_counts_long, by = c("age_group", "from_state")) %>%
  mutate(probability = count / total_count) %>%
  select(age_group, from_state, to_state, probability)




In [15]:
print(transition_probabilities)

# A tibble: 2,583 × 4
   age_group from_state to_state        probability
   <chr>     <chr>      <chr>                 <dbl>
 1 Age 0-04  DECEASED   DECEASED                  1
 2 Age 0-04  DECEASED   DSS                       0
 3 Age 0-04  DECEASED   DSS_EDU                   0
 4 Age 0-04  DECEASED   DSS_EDU_T1                0
 5 Age 0-04  DECEASED   DSS_EMPLOYED              0
 6 Age 0-04  DECEASED   DSS_EMPLOYED_T1           0
 7 Age 0-04  DECEASED   DSS_HOSPITAL              0
 8 Age 0-04  DECEASED   DSS_T1                    0
 9 Age 0-04  DECEASED   DSS_T2                    0
10 Age 0-04  DECEASED   DSS-OTHER                 0
# ℹ 2,573 more rows


## Step 3: Estimate New Entrants

1. Estimate the Total Population by Age Group:

In [30]:
# Assuming dss_pop_by_age contains the total population by age group for each year
dss_pop_by_age_summary <- dss_pop_by_age %>%
  group_by(age_group) %>%
  summarise(total_pop = sum(total_pop), .groups = 'drop')


In [24]:
# Rename new entrants columns
new_entrants <- new_entrants %>%
  rename(new_entrants = `Sum of new_entrants`)

2. Assume a Percentage for New Entrants:

In [25]:
names(new_entrants)

[1] "age_group"    "refyr"        "new_entrants"

In [26]:
# Assuming new_entrants contains the number of new entrants by age group for each year
new_entrants_summary <- new_entrants %>%
  group_by(age_group) %>%
  summarise(total_new_entrants = sum(new_entrants), .groups = 'drop')

# Calculate the percentage of new entrants
new_entrants_percentage <- new_entrants_summary %>%
  left_join(dss_pop_by_age_summary, by = "age_group") %>%
  mutate(percentage_new_entrants = total_new_entrants / total_pop)


3. Distribute New Entrants by State:

In [27]:
# Assuming the initial distribution of new entrants across states can be estimated from historical data
initial_state_distribution <- transition_counts_long %>%
  filter(from_state == "DSS") %>%
  group_by(age_group, to_state) %>%
  summarise(count = sum(count), .groups = 'drop') %>%
  mutate(probability = count / sum(count))

# Apply this distribution to estimate the number of new entrants by state
new_entrants_by_state <- new_entrants_summary %>%
  left_join(initial_state_distribution, by = "age_group") %>%
  mutate(new_entrants_by_state = total_new_entrants * probability) %>%
  select(age_group, to_state, new_entrants_by_state)



## Step 4: Model Initialization for Future Projection
1. Set Initial Population for Simulation

Use the population data from the last available year before 2021 (e.g., 2020) as your initial cohort.

In [28]:
initial_population_2020 <- dss_pop_by_age %>%
  filter(refyr == 2020) %>%
  select(age_group, total_pop)


2. Adjust for New Entrants:

In [31]:
# Calculate new entrants for 2023
new_entrants_2023 <- new_entrants_percentage %>%
  left_join(total_population_by_age, by = "age_group") %>%
  mutate(NewEntrants = round(TotalPopulation * NewEntrantsPercentage / 100, na.rm = TRUE))

# Distribute new entrants by state
new_entrants_distribution <- new_entrants_2023 %>%
  left_join(new_entrants_by_state, by = c("age_group")) %>%
  mutate(NewEntrantsByState = round(NewEntrants * Proportion, na.rm = TRUE))


ERROR: Error: object 'total_population_by_age' not found


## Step 5: Simulate Future States
1. Apply Transition Probabilities:

In [ ]:
# Apply transition probabilities to simulate future states
future_states <- initial_population %>%
  rename(from_state = state, InitialCount = total_pop) %>%
  left_join(transition_probabilities, by = "age_group") %>%
  mutate(FutureCount = InitialCount * TransitionProbability) %>%
  group_by(age_group, to_state) %>%
  summarize(FutureCount = sum(FutureCount))


2. Iterate Annually:

In [ ]:
# For each year, apply the transition probabilities and adjust the cohort by adding new entrants
future_states_2024 <- future_states %>%
  rename(from_state = to_state, InitialCount = FutureCount) %>%
  left_join(transition_probabilities, by = "age_group") %>%
  mutate(FutureCount = InitialCount * TransitionProbability) %>%
  group_by(age_group, to_state) %>%
  summarize(FutureCount = sum(FutureCount))

# Add new entrants to the future states
final_population_2024 <- future_states_2024 %>%
  left_join(new_entrants_distribution, by = c("age_group", "to_state" = "state")) %>%
  mutate(FinalCount = FutureCount + coalesce(NewEntrantsByState, 0)) %>%
  select(age_group, state = to_state, Count = FinalCount)
